# A YOLOv5 version of a CNN to train a sea turtle detection network

In [3]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from shutil import copyfile
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from pathlib import Path
import time
from IPython.display import display, clear_output

In [2]:
trainDf = pd.read_csv('Train.csv')
train, val = train_test_split(trainDf, test_size=0.2, random_state=42)

### Create directories

In [3]:
os.makedirs('yolov5_data/images/train', exist_ok=True)
os.makedirs('yolov5_data/images/val', exist_ok=True)
os.makedirs('yolov5_data/labels/train', exist_ok=True)
os.makedirs('yolov5_data/labels/val', exist_ok=True)

### Yolo format function

In [4]:
def convertToYoloFormat(df, imageSet):
    for idx, row in df.iterrows():
        imId = row['Image_ID']
        imgPath = f'IMAGES_512/{imId}.JPG'
        labelPath = f'yolov5_data/labels/{imageSet}/{imId}.txt'

        with open(labelPath, 'w') as f:
            width, height = 512, 512
            xCenter = row['x'] + row['w'] / 2
            yCenter = row['y'] + row['h'] / 2
            f.write(f"0 {xCenter} {yCenter} {row['w']} {row['h']}\n")

        if imageSet == 'train':
            copyfile(imgPath, f'yolov5_data/images/train/{imId}.JPG')
        else:
            copyfile(imgPath, f'yolov5_data/images/val/{imId}.JPG')

convertToYoloFormat(train, 'train')
convertToYoloFormat(val, 'val')

### Run and train the model

In [1]:
!python yolov5/train.py --img 512 --batch 16 --epochs 25 --data yolov5_data.yaml --weights yolov5l.pt


       0/24       6.3G    0.09633     0.0257          0         36        512:  /home/resteves/SeaTurtleFaceDetect/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
       0/24       6.3G    0.09575    0.02581          0         36        512:  /home/resteves/SeaTurtleFaceDetect/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
       0/24       6.3G      0.095    0.02586          0         30        512:  /home/resteves/SeaTurtleFaceDetect/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
       0/24       6.3G    0.09449    0.02596          0         35        512:  /home/resteves/SeaTurtleFaceDete

### Evaluate the model

In [4]:
from pathlib import Path

ss = pd.read_csv('SampleSubmission.csv')

testImagesDir = Path('IMAGES_512/')

!python yolov5/detect.py --weights yolov5/runs/train/exp2/weights/best.pt --source {testImagesDir} --img 512 --save-txt --save-conf --conf-thres 0.25

detect: weights=['yolov5/runs/train/exp2/weights/best.pt'], source=IMAGES_512, data=yolov5/data/coco128.yaml, imgsz=[512, 512], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-350-g6096750f Python-3.10.14 torch-2.4.0+cu121 CUDA:0 (NVIDIA T1200 Laptop GPU, 4096MiB)

Fusing layers... 
Model summary: 267 layers, 46108278 parameters, 0 gradients, 107.6 GFLOPs
image 1/2023 /home/resteves/SeaTurtleFaceDetect/IMAGES_512/00309D7D.JPG: 384x512 1 turtle_face, 42.1ms
image 2/2023 /home/resteves/SeaTurtleFaceDetect/IMAGES_512/00380F21.JPG: 384x512 1 turtle_face, 36.6ms
image 3/2023 /home/resteves/SeaTurtleFaceDetect/IMAGES_512/003B27CF.JPG: 384x512 1 turtle_face, 

# Run this cell for image evaluation

In [ ]:

# Load ground truth data
trainDf = pd.read_csv('Train.csv')

# Directory for test images
testImagesDir = Path('IMAGES_512/')

# Predictions directory
predictionsDir = Path('yolov5/runs/detect/exp2/labels')

# Output directory for the images with bounding boxes
outputDir = Path('output_images')
outputDir.mkdir(parents=True, exist_ok=True)

def drawDashedRectangle(draw, xy, outline, width=2, dash=(5, 5)):
    x0, y0, x1, y1 = xy
    for i in range(x0, x1, dash[0] + dash[1]):
        draw.line([(i, y0), (i + dash[0], y0)], fill=outline, width=width)
        draw.line([(i, y1), (i + dash[0], y1)], fill=outline, width=width)
    for i in range(y0, y1, dash[0] + dash[1]):
        draw.line([(x0, i), (x0, i + dash[0])], fill=outline, width=width)
        draw.line([(x1, i), (x1, i + dash[0])], fill=outline, width=width)

def drawBoundingBox(image, box, outline_color="red", width=4, dashed=False):
    draw = ImageDraw.Draw(image)
    x, y, w, h = box
    imw, imh = image.size
    xy = [int(x * imw), int(y * imh), int((x + w) * imw), int((y + h) * imh)]
    if dashed:
        drawDashedRectangle(draw, xy, outline=outline_color, width=width)
    else:
        draw.rectangle(xy, outline=outline_color, width=width)

def calculateIOU(boxA, boxB):
    # Convert to absolute coordinates
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[0] + boxA[2], boxB[0] + boxB[2])
    yB = min(boxA[1] + boxA[3], boxB[1] + boxB[3])

    # Compute the area of intersection
    interArea = max(0, xB - xA) * max(0, yB - yA)

    # Compute the area of both the prediction and ground-truth rectangles
    boxAArea = boxA[2] * boxA[3]
    boxBArea = boxB[2] * boxB[3]

    # Compute the intersection over union by taking the intersection area and dividing it by the sum of prediction + ground-truth areas - the intersection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    return iou

# Select 100 images to process
sampleImages = trainDf.sample(100)

for index, row in sampleImages.iterrows():
    imId = row['Image_ID']
    predFile = predictionsDir / f'{imId}.txt'

    imagePath = testImagesDir / f'{imId}.JPG'
    image = Image.open(imagePath)
    
    # Get ground truth box from train_df
    groundTruthBox = [row['x'], row['y'], row['w'], row['h']]

    # Draw the predicted bounding box (solid line)
    if predFile.exists():
        with open(predFile) as f:
            line = f.readline().strip().split()
            if line:
                _, x_center, y_center, width, height, _ = map(float, line)
                predBox = [x_center - width / 2, y_center - height / 2, width, height]
                drawBoundingBox(image, predBox, outline_color="red", width=4)

                # Calculate IoU
                iou = calculateIOU(groundTruthBox, predBox)
                print(f"Image {imId}: IoU = {iou:.4f}")

    # Draw the ground truth bounding box (dashed line)
    drawBoundingBox(image, groundTruthBox, outline_color="green", width=4, dashed=True)

        # Clear the previous output
    clear_output(wait=True)

    # Display the current image
    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    plt.title(f"Image: {imId} - Green: Ground Truth (Dashed), Red: Prediction (Solid)")
    plt.axis('off')
    display(plt.gcf())

    # Save the image with bounding boxes
    outputImagePath = outputDir / f"{imId}_boxed.JPG"
    image.save(outputImagePath)
    time.sleep(0.3)

    print(f"Saved {outputImagePath}")

print("Processing complete.")

# Run this cell for Sample submission

In [11]:
# Load sample submission
ss = pd.read_csv('SampleSubmission.csv')

testImagesDir = Path('IMAGES_512/')

# Inference command
#!python yolov5/detect.py --weights yolov5/runs/train/exp2/weights/best.pt --source {test_images_dir} --img 512 --save-txt --save-conf --conf-thres 0.25

# Parse predictions and update submission file
predictionsDir = Path('yolov5/runs/detect/exp7/labels/')
preds = []

for imId in ss['Image_ID']:
    predFile = predictionsDir / f'{imId}.txt'
    if predFile.exists():
        with open(predFile) as f:
            line = f.readline().strip().split()
            if line:
                _, x_center, y_center, width, height, _ = map(float, line)
                preds.append([x_center - width / 2, y_center - height / 2, width, height])
            else:
                preds.append([0, 0, 0, 0])
    else:
        preds.append([0, 0, 0, 0])

ss[['x', 'y', 'w', 'h']] = preds
ss.to_csv('Submission3.csv', index=False)